# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [6]:
#Your code here; use a subquery. No join will be necessary.
c.execute(""" select customerNumber, contactLastName, contactFirstName from customers 
               where customerNumber in (select customerNumber from orders where orderDate = '2003-01-31'); """)
df_subquery = pd.DataFrame(c.fetchall())
df_subquery.columns = [x[0] for x in c.description]
df_subquery

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [12]:
#Your code here
c.execute(""" select productName, count(distinct orderNumber) as num_orders, sum(quantityOrdered) as total_sold from 
               products join orderDetails using(productCode) 
               group by 1
               order by total_sold desc""")
df_total = pd.DataFrame(c.fetchall())
df_total.columns = [x[0] for x in c.description]
df_total.head()

,productName,num_orders,total_sold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [19]:
#Your code here
c.execute(""" select productName, count(distinct customerNumber) total_people_ordered from
              products join
              (orderDetails join orders using(orderNumber))
              using(productCode)
              group by 1
              order by total_people_ordered desc""")
df_people_total = pd.DataFrame(c.fetchall())
df_people_total.columns = [x[0] for x in c.description]
df_people_total.head()

,productName,total_people_ordered
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


**MEG:** Just looked at the solution and understand the multiple joins method, but why does this not work if you don't group by 1? Otherwise it only displays 1 answer.

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [23]:
#Your code here
c.execute("""select distinct employeeNumber, officeCode, o.city, firstName, lastName
                    from employees e join offices o using(officeCode)
                    join customers c on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders using (customerNumber)
                    join orderdetails using (orderNumber)
                    where productCode in (select productCode
                                                 from products join orderdetails using (productCode)
                                                 join orders using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20); """)
df_prod_ordered_by_under_20 = pd.DataFrame(c.fetchall())
df_prod_ordered_by_under_20.columns = [x[0] for x in c.description]
df_prod_ordered_by_under_20

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur
5,1286,3,NYC,Foon Yue,Tseng
6,1166,1,San Francisco,Leslie,Thompson
7,1165,1,San Francisco,Leslie,Jennings
8,1612,6,Sydney,Peter,Marsh
9,1188,2,Boston,Julie,Firrelli


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [34]:
#Your code here
c.execute(""" select employeeNumber, lastName, firstName, count(customerNumber) as Number_customers from
              employees e join customers
              on e.employeeNumber = customers.salesRepEmployeeNumber
              where salesRepEmployeeNumber in 
              (select salesRepEmployeeNumber from customers group by 1
              having avg(creditLimit) > 15000) 
              group by 1
              order by Number_customers desc, lastName asc, firstName asc""")
df_15K_credit = pd.DataFrame(c.fetchall())
df_15K_credit.columns = [x[0] for x in c.description]
df_15K_credit

,employeeNumber,lastName,firstName,Number_customers
0,1401,Castillo,Pamela,10
1,1504,Jones,Barry,9
2,1501,Bott,Larry,8
3,1323,Vanauf,George,8
4,1370,Hernandez,Gerard,7
5,1286,Tseng,Foon Yue,7
6,1337,Bondur,Loui,6
7,1188,Firrelli,Julie,6
8,1702,Gerard,Martin,6
9,1165,Jennings,Leslie,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!